# **Import and model change**

In [2]:
!pip install segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.2 MB/s eta 0:00:00


In [15]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

In [17]:
from segmentation_models import Unet
from keras.layers import Input, Conv2D
from keras.models import Model
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from segmentation_models.losses import DiceLoss
from segmentation_models.metrics import FScore

In [5]:
base_model = sm.Linknet(backbone_name='vgg16', classes=4, encoder_weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
#base_model = sm.Unet(backbone_name='resnet34', encoder_weights='imagenet', classes=4)

In [6]:
inp = Input(shape=(128, 128, 4))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = base_model(l1)

In [7]:
model = Model(inp, out, name=base_model.name)

In [8]:
import keras
import tensorflow as tf
from tensorflow.keras.backend import argmax
import matplotlib.pyplot as plt

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data**

In [11]:
import numpy as np

In [12]:
train_dataset_path = '/content/drive/MyDrive/Progetto DeepL/train_imgs_2D (1).npy'
train_mask_path = '/content/drive/MyDrive/Progetto DeepL/train_labels_2D (1).npy'
val_dataset_path = '/content/drive/MyDrive/Progetto DeepL/validation_imgs_2D (1).npy'
val_mask_path = '/content/drive/MyDrive/Progetto DeepL/validation_labels_2D (1).npy'

X_train = np.load(train_dataset_path)
Y_train = np.load(train_mask_path)
X_val = np.load(val_dataset_path)
Y_val = np.load(val_mask_path)

In [13]:
X_val.shape

(927, 128, 128, 4)

# **Training**

In [18]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=sm.losses.DiceLoss(), metrics=[sm.metrics.FScore(), sm.metrics.IOUScore()])

In [19]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]

In [20]:
history = model.fit(x=X_train, y= Y_train, epochs=30, batch_size=32, verbose=1, validation_data=(X_val, Y_val), callbacks=[callbacks])

Epoch 1/30
116/116 [==============================] - 81s 440ms/step - loss: 0.5661 - f1-score: 0.4340 - iou_score: 0.3362 - val_loss: 0.5746 - val_f1-score: 0.4253 - val_iou_score: 0.3335
Epoch 2/30
116/116 [==============================] - 28s 245ms/step - loss: 0.3799 - f1-score: 0.6202 - iou_score: 0.4883 - val_loss: 0.4454 - val_f1-score: 0.5544 - val_iou_score: 0.4538
Epoch 3/30
116/116 [==============================] - 27s 233ms/step - loss: 0.3234 - f1-score: 0.6766 - iou_score: 0.5435 - val_loss: 0.4926 - val_f1-score: 0.5073 - val_iou_score: 0.4081
Epoch 4/30
116/116 [==============================] - 27s 233ms/step - loss: 0.2864 - f1-score: 0.7136 - iou_score: 0.5825 - val_loss: 0.4442 - val_f1-score: 0.5556 - val_iou_score: 0.4586
Epoch 5/30
116/116 [==============================] - 28s 241ms/step - loss: 0.2643 - f1-score: 0.7357 - iou_score: 0.6072 - val_loss: 0.4149 - val_f1-score: 0.5849 - val_iou_score: 0.4837
Epoch 6/30
116/116 [==============================] - 2

In [23]:
import pandas as pd
epochs_table = pd.DataFrame()
epochs_table['Loss'] = history.history['loss']
epochs_table['Validation Loss'] = history.history['val_loss']
epochs_table['Jaccard'] = history.history['iou_score']
epochs_table['Validation Jaccard'] = history.history['iou_score']
epochs_table['Dice score'] = history.history['f1-score']
epochs_table['Validation Dice score'] = history.history['val_f1-score']

epochs_table
epochs_table.to_csv('train_epochs_linknet.csv')

In [25]:
model.save('/content/drive/MyDrive/Progetto DeepL/models/model_linknet.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
model.save_weights('/content/drive/MyDrive/Progetto DeepL/models/model_linknet.weights.h5')

In [24]:
import os
os.path.getsize("model_linknet.h5")

244176416